In [1]:
import simpy
import random

In [2]:
# Cantidad de mesas-casetas de vacunacion dentro del recinto.
CANTIDAD_MESA = 20
# Espera Obligatoria.
ESPERA_OBLIGATORIA = 20
# Tiempo de la simulacion 8H = 480 M
TIEMPO_LABORAL = 480
#Generamos una fila
FILA_INGRESO=20

# Tiempo vacunacion 
TIEMPO_VACUNACION = 0

PERSONAS=0
# Tiempo Vacunación
PERSONA_TIEMPO_VACUNACION = 0
# Dicciccionario con los tiempo de vacunación por persona
DICCIONARIO_PERSONA_TIEMPO_VACUNACION = {}
# Diccionario con los tiempos de espera por persona.
DICCIONARIO_TIEMPO_DE_ESPERA = {}
# Diccionario con los Personas No vacunadas.
DICCIONARIO_PERSONAS_NO_VACUNADAS = {}

In [3]:
# Obtener el tiempo total que le toma a una persona hasta ser vacunada.  
def tiempo_hasta_vacunacion(persona, tiempoPersona):
     # Dicciccionario con los tiempo de vacunación por persona
    global DICCIONARIO_PERSONA_TIEMPO_VACUNACION
    DICCIONARIO_PERSONA_TIEMPO_VACUNACION[int(persona)] = tiempoPersona

In [4]:
# Obtener el tiempo total de espera de una persona
def tiempo_espera_vacunacion(persona, tiempoEspera):
    # Diccionario el tiempo de espera por persona.
    global DICCIONARIO_TIEMPO_DE_ESPERA
    DICCIONARIO_TIEMPO_DE_ESPERA[int(persona)] = tiempoEspera

In [5]:
class Principal(object):

    def __init__(self, environment,esperaObligatoria, cantMesas, tiempoVacunacion):
        
        # Guardamos como variable el entorno de ejecucion
        self.env = environment
        
        # Variable para el tiempo de espera obligatorio.
        self.esperaObligatoria = esperaObligatoria
        
        #Creamos el recurso que representa las mesas dentro del recinto
        self.mesas = simpy.Resource(environment, cantMesas)
        
        # Variable para el tiempo de vacunacion
        self.tiempoVacunacion = tiempoVacunacion
        
    #Funcion de Espera Obligatoria    
    def esperarObligatoria(self):
        yield self.env.timeout(self.esperaObligatoria)
        
    #Funcion random del tiempo que le toma realizar la vacunacion.
    def vacunar(self, persona):
        tiempo_randomico=random.randint(5, 10)
        yield self.env.timeout(tiempo_randomico)
        
    #Funcion de vacunacion por persona    
    def certificado(self):
        tiempo_randomico=random.randint(2, 3)
        yield self.env.timeout(tiempo_randomico)
        

In [6]:
def recinto_borja(env, personaId, gene_vacunacion):
    # Usamos el reloj de la simulacion (env.now()) para indicar la llegada cada persona.  
    print('Inicio de la persona [%s] llega al recinto Borja  %.2f.' % (personaId, env.now))
  
    # Guardamos EL Tiempo de inicio para calcular el tiempo que nos tomara la vacunacion por persona.
    inicioEspera = env.now
        
    #Aleatorio de personas que ingresan a vacunarse 
    
    # De la cola sacamos un total de 17 a 35 personas que  no cumplen o tienen problemas para ser vacunada
    control=random.randint(5, 10)
    #print(control)
    if random.randint(1,100) <= control :
        #print("Guardar Vacuna De :",personaId[personaId.find(" ")+1:len(personaId)],"presenta problemas De Salud Tiempo")
        DICCIONARIO_PERSONAS_NO_VACUNADAS[personaId[personaId.find(" ")+1:len(personaId)]] = (personaId, env.now)
        print('La persona [%s] no puede vacunarse '%(personaId))
        
    else:
        
        # Especificamos el uso de una mesa disponible con request().
        with gene_vacunacion.mesas.request() as mesa:
            #global VACUNADOS
            # Iniciamos el tiempo apenas ingresa.
            inicio = env.now
            
            # Agregramos a la persona en una mesa disponible
            yield mesa
            print('La persona [%s] pasa a la mesa de vacunacion : %.2f.' % (personaId, env.now))
            
    
            # Realizamos la vacunacion
            yield env.process(gene_vacunacion.vacunar(personaId))
            
            
            # Mandamos a guardar la informacion del tiempo que le tomo la vacunacion a la persona.
            tiempo_hasta_vacunacion(personaId, env.now-inicio)
            
            
            print('La persona [%s] fue vacunada a las %.2f.' % (personaId, env.now)) 
            
            

            #Realizamos el check de salud con una espera de 20m
            yield env.process(gene_vacunacion.esperarObligatoria())
            
            
            print('La persona [%s] pasa a recibir su certificado a las %.2f.' % (personaId, env.now))
            #Realizamos la generacion del certificado.
            yield env.process(gene_vacunacion.certificado())
            
            # La persona despues de ser vacuna sale del centro de vacunación
            print('Salida de la persona [%s] sale del recinto a las %.2f.' % (personaId, env.now)) 
            tiempo_espera_vacunacion(personaId, env.now-inicioEspera)
            print("Tiempo Total De",personaId,"=",env.now-inicioEspera)

In [7]:
def ejecutar_simulacion(env,esperaobligatoria, controlmesa,tiempoVacunacion):
    gene_vacunacion=Principal(env,esperaobligatoria, controlmesa,tiempoVacunacion)
    global PERSONAS 
    
    # Creamos fila de  20 persona que llegan y esperan afuera hasta que abran las puertas.
    for i in range(FILA_INGRESO):
        env.process(recinto_borja(env, '%d'%(i+1), gene_vacunacion))
        PERSONAS =PERSONAS+1
    
    
    # Ejecutamos la simulacion
    while True:
        #Intervalo Llegada
        yield env.timeout(random.randint(1,5))
        i+=1
        PERSONAS=PERSONAS+1
        #print("Total:",PERSONAS)
        env.process(recinto_borja(env,'%d'%(i+1), gene_vacunacion))  
    

print('Simulacion Vacunacion Ecuador')

# Inicializamos la semilla aleatoria
random.seed(77)

# Creamos el entorno de simulacion
env=simpy.Environment()
env.process(ejecutar_simulacion(env,ESPERA_OBLIGATORIA,CANTIDAD_MESA,TIEMPO_VACUNACION))

# Ejecutamos el proceso durante el tiempo de simulacion
env.run(until = TIEMPO_LABORAL)

Simulacion Vacunacion Ecuador
Inicio de la persona [1] llega al recinto Borja  0.00.
Inicio de la persona [2] llega al recinto Borja  0.00.
Inicio de la persona [3] llega al recinto Borja  0.00.
Inicio de la persona [4] llega al recinto Borja  0.00.
Inicio de la persona [5] llega al recinto Borja  0.00.
Inicio de la persona [6] llega al recinto Borja  0.00.
Inicio de la persona [7] llega al recinto Borja  0.00.
Inicio de la persona [8] llega al recinto Borja  0.00.
La persona [8] no puede vacunarse 
Inicio de la persona [9] llega al recinto Borja  0.00.
La persona [9] no puede vacunarse 
Inicio de la persona [10] llega al recinto Borja  0.00.
Inicio de la persona [11] llega al recinto Borja  0.00.
Inicio de la persona [12] llega al recinto Borja  0.00.
Inicio de la persona [13] llega al recinto Borja  0.00.
Inicio de la persona [14] llega al recinto Borja  0.00.
Inicio de la persona [15] llega al recinto Borja  0.00.
Inicio de la persona [16] llega al recinto Borja  0.00.
Inicio de la 